In [13]:
from statistics import mean
from functools import partial
import gymnasium as gym
from gymnasium.wrappers import TimeLimit

from env_hiv import HIVPatient

import torch
import torch.nn as nn
import numpy as np
from collections import deque
import random

from matplotlib import pyplot as plt
from copy import deepcopy

from datetime import datetime

In [14]:
env = TimeLimit(
    env=HIVPatient(domain_randomization=False), max_episode_steps=200
)

# Constants
NB_EPISODES = 500
GAMMA = 0.99
EPS_START = 1
EPS_END = 0.05
EPS_DECAY = 20000
EPS_LAG = 500
REPLAY_BUFFER_SIZE = 1000000
BATCH_SIZE = 1024
TARGET_UPDATE_PERIOD = 1000

# Seed for reproducibility
random.seed(0)
torch.manual_seed(0)

In [15]:
class ReplayBuffer:
    def __init__(self, capacity, device):
        self.capacity = capacity # capacity of the buffer
        self.data = []
        self.index = 0 # index of the next cell to be filled
        self.device = device
    def append(self, s, a, r, s_, d):
        if len(self.data) < self.capacity:
            self.data.append(None)
        self.data[self.index] = (s, a, r, s_, d)
        self.index = (self.index + 1) % self.capacity
    def sample(self, batch_size):
        batch = random.sample(self.data, batch_size)
        return list(map(lambda x:torch.Tensor(np.array(x)).to(self.device), list(zip(*batch))))
    def __len__(self):
        return len(self.data)

In [16]:
class dqn_agent:
    def __init__(self, config):
        self.device = config['device']
        self.nb_actions = config['nb_actions']
        self.gamma = config['gamma'] if 'gamma' in config.keys() else 0.95
        self.batch_size = config['batch_size'] if 'batch_size' in config.keys() else 100
        buffer_size = config['buffer_size'] if 'buffer_size' in config.keys() else int(1e5)
        self.memory = ReplayBuffer(buffer_size, self.device)
        self.epsilon_max = config['epsilon_max'] if 'epsilon_max' in config.keys() else 1.
        self.epsilon_min = config['epsilon_min'] if 'epsilon_min' in config.keys() else 0.01
        self.epsilon_stop = config['epsilon_decay_period'] if 'epsilon_decay_period' in config.keys() else 1000
        self.epsilon_delay = config['epsilon_delay_decay'] if 'epsilon_delay_decay' in config.keys() else 20
        self.epsilon_step = (self.epsilon_max-self.epsilon_min)/self.epsilon_stop
        self.model = torch.nn.Sequential(nn.Linear(env.observation_space.shape[0], config['nb_neurons']),
                          nn.ReLU(),
                          nn.Linear(config['nb_neurons'], config['nb_neurons']),
                          nn.ReLU(), 
                          nn.Linear(config['nb_neurons'], config['nb_neurons']),
                          nn.ReLU(), 
                          nn.Linear(config['nb_neurons'], config['nb_neurons']),
                          nn.ReLU(), 
                          nn.Linear(config['nb_neurons'], config['nb_neurons']),
                          nn.ReLU(), 
                          nn.Linear(config['nb_neurons'], env.action_space.n)).to(self.device) 
        self.target_model = deepcopy(self.model).to(self.device)
        self.criterion = config['criterion'] if 'criterion' in config.keys() else torch.nn.MSELoss()
        lr = config['learning_rate'] if 'learning_rate' in config.keys() else 0.001
        self.optimizer = config['optimizer'] if 'optimizer' in config.keys() else torch.optim.Adam(self.model.parameters(), lr=lr)
        self.nb_gradient_steps = config['gradient_steps'] if 'gradient_steps' in config.keys() else 1
        self.update_target_strategy = config['update_target_strategy'] if 'update_target_strategy' in config.keys() else 'replace'
        self.update_target_freq = config['update_target_freq'] if 'update_target_freq' in config.keys() else 20
        self.update_target_tau = config['update_target_tau'] if 'update_target_tau' in config.keys() else 0.005
    
    def greedy_action(self, state):
        with torch.no_grad():
            Q = self.model(torch.Tensor(state).unsqueeze(0).to(self.device))
        return torch.argmax(Q).item()
    
    def gradient_step(self):
        if len(self.memory) > self.batch_size:
            X, A, R, Y, D = self.memory.sample(self.batch_size)
            QYmax = self.target_model(Y).max(1)[0].detach()
            update = torch.addcmul(R, 1-D, QYmax, value=self.gamma)
            QXA = self.model(X).gather(1, A.to(torch.long).unsqueeze(1))
            loss = self.criterion(QXA, update.unsqueeze(1))
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step() 

    def save(self, path):
        # Save the model on the CPU
        torch.save(self.model.to('cpu').state_dict(), path)
        self.model.to(self.device)
    
    def load(self, path):
        self.model.load_state_dict(torch.load(path, map_location=self.device))
        self.target_model.load_state_dict(self.model.state_dict())
        
        # When we load the model we just use it for evaluation so we make sure the actions are fuly driven by the model
        self.epsilon_max = 0
        self.epsilon_min = 0

    def act(self, observation, use_random=False):
        if use_random:
            action = env.action_space.sample()
        else:
            action = self.greedy_action(observation)
        return action

    def train(self, env, max_episode):
        episode_return = []
        episode = 0
        episode_cum_reward = 0
        state, _ = env.reset()
        epsilon = self.epsilon_max
        step = 0
        while episode < max_episode:
            # update epsilon
            if step > self.epsilon_delay:
                epsilon = max(self.epsilon_min, epsilon-self.epsilon_step)

            action = self.act(state, np.random.rand() < epsilon)

            # step
            next_state, reward, done, trunc, _ = env.step(action)
            self.memory.append(state, action, reward, next_state, done)
            episode_cum_reward += reward
            # train
            for _ in range(self.nb_gradient_steps): 
                self.gradient_step()
            # update target network if needed
            if self.update_target_strategy == 'replace':
                if step % self.update_target_freq == 0: 
                    self.target_model.load_state_dict(self.model.state_dict())
            if self.update_target_strategy == 'ema':
                target_state_dict = self.target_model.state_dict()
                model_state_dict = self.model.state_dict()
                tau = self.update_target_tau
                for key in model_state_dict:
                    target_state_dict[key] = tau*model_state_dict[key] + (1-tau)*target_state_dict[key]
                self.target_model.load_state_dict(target_state_dict)

            # next transition
            step += 1
            
            if done or trunc:
                episode += 1
                print("Episode ", '{:3d}'.format(episode), 
                      ", epsilon ", '{:6.2f}'.format(epsilon), 
                      ", batch size ", '{:5d}'.format(len(self.memory)), 
                      ", episode return ", '{:4.1f}'.format(episode_cum_reward),
                      sep='')
                state, _ = env.reset()

                # Save the model if it is the best so far
                if len(episode_return) == 0 or episode_cum_reward >= max(episode_return):
                    print("New best model saved")
                    now = datetime.now().strftime("%Y%m%d-%H%M%S")
                    self.save(f"best_model_{now}.pt")

                episode_return.append(episode_cum_reward)

                episode_cum_reward = 0
                if episode % 10 == 0:
                    plt.plot(episode_return)
                    plt.xlabel('Episode')
                    plt.ylabel('Return')
                    plt.title('Return per Episode')
                    plt.show()
            else:
                state = next_state
        return episode_return

In [17]:
class dqn_agent_with_memory:
    def __init__(self, config):
        self.device = config['device']
        self.nb_actions = config['nb_actions']
        self.gamma = config['gamma'] if 'gamma' in config.keys() else 0.95
        self.batch_size = config['batch_size'] if 'batch_size' in config.keys() else 100
        buffer_size = config['buffer_size'] if 'buffer_size' in config.keys() else int(1e5)
        self.replay_buffer = ReplayBuffer(buffer_size, self.device)
        self.epsilon_max = config['epsilon_max'] if 'epsilon_max' in config.keys() else 1.
        self.epsilon_min = config['epsilon_min'] if 'epsilon_min' in config.keys() else 0.01
        self.epsilon_stop = config['epsilon_decay_period'] if 'epsilon_decay_period' in config.keys() else 1000
        self.epsilon_delay = config['epsilon_delay_decay'] if 'epsilon_delay_decay' in config.keys() else 20
        self.epsilon_step = (self.epsilon_max-self.epsilon_min)/self.epsilon_stop
        self.memory_length = config['memory_length'] if 'memory_length' in config.keys() else 10
        self.memory = deque([-1 for _ in range(env.observation_space.shape[0] + self.memory_length)], maxlen=self.memory_length)
        self.model = torch.nn.Sequential(nn.Linear(env.observation_space.shape[0] + self.memory_length, config['nb_neurons']),
                          nn.ReLU(),
                          nn.Linear(config['nb_neurons'], config['nb_neurons']),
                          nn.ReLU(), 
                          nn.Linear(config['nb_neurons'], config['nb_neurons']),
                          nn.ReLU(), 
                          nn.Linear(config['nb_neurons'], config['nb_neurons']),
                          nn.ReLU(), 
                          nn.Linear(config['nb_neurons'], config['nb_neurons']),
                          nn.ReLU(), 
                          nn.Linear(config['nb_neurons'], env.action_space.n)).to(self.device) 
        self.target_model = deepcopy(self.model).to(self.device)
        self.criterion = config['criterion'] if 'criterion' in config.keys() else torch.nn.MSELoss()
        lr = config['learning_rate'] if 'learning_rate' in config.keys() else 0.001
        self.optimizer = config['optimizer'] if 'optimizer' in config.keys() else torch.optim.Adam(self.model.parameters(), lr=lr)
        self.nb_gradient_steps = config['gradient_steps'] if 'gradient_steps' in config.keys() else 1
        self.update_target_strategy = config['update_target_strategy'] if 'update_target_strategy' in config.keys() else 'replace'
        self.update_target_freq = config['update_target_freq'] if 'update_target_freq' in config.keys() else 20
        self.update_target_tau = config['update_target_tau'] if 'update_target_tau' in config.keys() else 0.005
    
    def normalize_state(self, state):
        normalization_factors = torch.Tensor([1e5, 1e4, 1e1, 1e4, 1e1, 1e1] + [1] * self.memory_length).to(self.device)
        state = state / normalization_factors   
        return state

    def greedy_action(self, state):
        with torch.no_grad():
            state_and_memory = torch.Tensor(np.concatenate((state, self.memory))).unsqueeze(0).to(self.device)
            state_and_memory = self.normalize_state(state_and_memory)
            Q = self.model(state_and_memory)
        return torch.argmax(Q).item()
    
    def gradient_step(self):
        if len(self.replay_buffer) > self.batch_size:
            X, A, R, Y, D = self.replay_buffer.sample(self.batch_size)
            X = self.normalize_state(X)
            Y = self.normalize_state(Y)
            QYmax = self.target_model(Y).max(1)[0].detach()
            update = torch.addcmul(R, 1-D, QYmax, value=self.gamma)
            QXA = self.model(X).gather(1, A.to(torch.long).unsqueeze(1))
            loss = self.criterion(QXA, update.unsqueeze(1))
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step() 

    def save(self, path):
        # Save the model on the CPU
        torch.save(self.model.to('cpu').state_dict(), path)
        self.model.to(self.device)

    def load(self, path):
        self.model.load_state_dict(torch.load(path, map_location=self.device))
        self.target_model.load_state_dict(self.model.state_dict())
        
        # When we load the model we just use it for evaluation so we make sure the actions are fuly driven by the model
        self.epsilon_max = 0
        self.epsilon_min = 0

    def act(self, observation, use_random=False):
        if use_random:
            action = env.action_space.sample()
        else:
            action = self.greedy_action(observation)
        self.memory.append(action)
        return action

    def train(self, env, max_episode):
        episode_return = []
        episode = 0
        episode_cum_reward = 0
        state, _ = env.reset()
        epsilon = self.epsilon_max
        step = 0
        while episode < max_episode:
            # update epsilon
            if step > self.epsilon_delay:
                epsilon = max(self.epsilon_min, epsilon-self.epsilon_step)

            action = self.act(state, np.random.rand() < epsilon)

            # step
            next_state, reward, done, trunc, _ = env.step(action)
            self.replay_buffer.append(np.concatenate((state, self.memory)), action, reward, np.concatenate((next_state, self.memory)), done)
            episode_cum_reward += reward
            # train
            for _ in range(self.nb_gradient_steps): 
                self.gradient_step()
            # update target network if needed
            if self.update_target_strategy == 'replace':
                if step % self.update_target_freq == 0: 
                    self.target_model.load_state_dict(self.model.state_dict())
            if self.update_target_strategy == 'ema':
                target_state_dict = self.target_model.state_dict()
                model_state_dict = self.model.state_dict()
                tau = self.update_target_tau
                for key in model_state_dict:
                    target_state_dict[key] = tau*model_state_dict[key] + (1-tau)*target_state_dict[key]
                self.target_model.load_state_dict(target_state_dict)

            # next transition
            step += 1
            
            if done or trunc:
                episode += 1
                print("Episode ", '{:3d}'.format(episode), 
                      ", epsilon ", '{:6.2f}'.format(epsilon), 
                      ", batch size ", '{:5d}'.format(len(self.replay_buffer)), 
                      ", episode return ", '{:4.1f}'.format(episode_cum_reward),
                      sep='')
                state, _ = env.reset()

                # Save the model if it is the best so far
                if len(episode_return) == 0 or episode_cum_reward >= max(episode_return):
                    print("New best model saved")
                    now = datetime.now().strftime("%Y%m%d-%H%M%S")
                    self.save(f"best_model_{now}.pt")

                episode_return.append(episode_cum_reward)

                episode_cum_reward = 0
                if episode % 10 == 0:
                    plt.plot(episode_return)
                    plt.xlabel('Episode')
                    plt.ylabel('Return')
                    plt.title('Return per Episode')
                    plt.show()
            else:
                state = next_state
        return episode_return

In [18]:
# Declare network
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

state_dim = env.observation_space.shape[0]
n_action = env.action_space.n


# DQN config
config = {'nb_actions': n_action,
          'learning_rate': 0.001,
          'gamma':0.99,
          'buffer_size': 1000000,
          'epsilon_min': 0.05,
          'epsilon_max': 1.,
          'epsilon_decay_period': 15000,#10000,
          'epsilon_delay_decay': 1000,#300,
          'batch_size': 1024,
          'gradient_steps': 5,
          'update_target_strategy': 'ema',#'replace', # or 'ema'
          'update_target_freq': 200,
          'update_target_tau': 0.001,
          'criterion': torch.nn.SmoothL1Loss(),
          'nb_neurons': 256,
          'device': device,
          'memory_length':5}

# Train agent
agent = dqn_agent(config)
# agent = dqn_agent_with_memory(config)
scores = agent.train(env, 500)
plt.plot(scores)

Episode   1, epsilon   1.00, batch size   200, episode return 8411206.0
New best model saved


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [ ]:
path_to_trained_model = "best_model_20211018-191153.pt"
agent = dqn_agent_with_memory(config)
agent.load(path_to_trained_model)

def evaluate_agent(agent: dqn_agent_with_memory, env: gym.Env, nb_episode: int = 10) -> float:
    """
    Evaluate an agent in a given environment.

    Args:
        agent (Agent): The agent to evaluate.
        env (gym.Env): The environment to evaluate the agent in.
        nb_episode (int): The number of episode to evaluate the agent.

    Returns:
        float: The mean reward of the agent over the episodes.
    """
    rewards: list[float] = []
    for _ in range(nb_episode):
        obs, info = env.reset()
        done = False
        truncated = False
        episode_reward = 0
        while not done and not truncated:
            action = agent.act(obs)
            obs, reward, done, truncated, _ = env.step(action)
            episode_reward += reward
        rewards.append(episode_reward)
    return mean(rewards)


evaluate_HIV = partial(
    evaluate_agent, env=TimeLimit(HIVPatient(), max_episode_steps=200)
)

score_agent: float = evaluate_HIV(agent=agent, nb_episode=1)